In [28]:
df = pd.read_csv('data/evidence.csv',sep=',')

In [29]:
df.tail()

,TX_ID,BLOCK_ID,BLOCK_TIMESTAMP,SENDER,MSG_KEY,MSG_VALUE,ASSET1_EXPONENT,ASSET2_EXPONENT,ASSET1_SYMBOL,ASSET1_AMOUNT,ASSET2_SYMBOL,ASSET2_AMOUNT,ASSET1_PRICE,ASSET2_PRICE
12105,6E3D06B5DED2E53D210E1CC86DC0953366C70702D58B45...,4712330,2022-06-08 01:32:12.199,osmo18qx59wy8s3ytax3e0akna934e86mw776vlzjtq,pool_id,704.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12106,0711AFF7C1828299F527854D6C543C75A729FD09ECC39F...,4712884,2022-06-08 02:29:39.413,osmo1rejv949h6k78u7fyeudv3lqgveqpfr0du2nwlq,pool_id,585.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12107,4F79D10FD5521401A2D61C72BDE561415A06F7DCF8AAC2...,4711478,2022-06-08 00:03:40.603,osmo1yfq7eu4rx9u49gp5td6xu0c7kw5qh5eg8yr4pa,pool_id,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12108,EEC736070E392A2E5354C02F10EDC24F8C28121C0C6583...,4711850,2022-06-08 00:42:27.584,osmo10rgny3xd5fvg02wnmy4h9zuktnfthvrlwdjwf5,pool_id,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12109,90A07330611A71C38246452D36DC0F0ADA1213F9A91049...,4712235,2022-06-08 01:22:23.572,osmo1yglld3aary7lnrrn2dz7la84kmnmen4kpsxzay,pool_id,712.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
import pandas as pd
def load_evidence():
    df = pd.read_csv('data/evidence.csv',sep=',')
    #"cols = df.columns[0].split(',')
    #"df = df[df.columns[0]].str.split(',',expand=True).iloc[:,0:len(cols)]
    #"df.columns = cols
    df2 = df.query('MSG_KEY=="pool_id"').copy()
    df2['MSG_VALUE'] = df2['MSG_VALUE'].astype('int').copy()
    df = df.drop(df2.index)
    df = df.drop('MSG_VALUE',axis=1)
    df['BLOCK_TIMESTAMP'] = pd.to_datetime(df['BLOCK_TIMESTAMP'])
    df['ASSET1_USD'] = df['ASSET1_AMOUNT'] * df['ASSET1_PRICE']
    df['ASSET2_USD'] = df['ASSET2_AMOUNT'] * df['ASSET2_PRICE']

    df['dummy'] = 1
    df['POOL_ID'] = df['TX_ID'].map(df2.groupby('TX_ID')['MSG_VALUE'].first())
    df['USD'] = df['ASSET1_USD'] + df['ASSET2_USD']
    return df

df = load_evidence()

df.head()

,TX_ID,BLOCK_ID,BLOCK_TIMESTAMP,SENDER,MSG_KEY,ASSET1_EXPONENT,ASSET2_EXPONENT,ASSET1_SYMBOL,ASSET1_AMOUNT,ASSET2_SYMBOL,ASSET2_AMOUNT,ASSET1_PRICE,ASSET2_PRICE,ASSET1_USD,ASSET2_USD,dummy,POOL_ID,USD
0,BE7C1F80D781CDA3BF87F3CA2607DEEC9BF3C4DDECBD49...,4709565,2022-06-07 20:45:23.383,osmo1qnqwxazzha6lsx2rs2gewu2337uha5lz5uf2yt,tokens_out,6.0,6.0,CRBRUS,2557.430883,HUAHUA,522.867991,0.000011,0.000047,0.027550,0.024391,1,658,0.051942
1,BC2F40C96EF02403A11B206E77B8EAB2BB0F0F65660D40...,4708075,2022-06-07 18:09:53.084,osmo16mdmh3lfphnmqzn20c59x75nfhfktfz9wexzep,tokens_out,6.0,6.0,CRBRUS,380852.497456,HUAHUA,79554.783095,0.000011,0.000047,4.102815,3.711151,1,658,7.813966
2,DBD856249340F8707CD54DD9C7E2605C9E01A31904A682...,4711580,2022-06-08 00:14:14.320,osmo1wfyaqhxsczc4mlgk54trxznmqkld09p79phzqn,tokens_out,6.0,6.0,ATOM,1.369754,HUAHUA,263978.971856,8.887505,0.000047,12.173696,12.314354,1,606,24.488050
3,2231E194F2EFA3F5818FF6D6E03C13BCEA9FF3312DF6C8...,4712865,2022-06-08 02:27:42.386,osmo1adtxg4klr2z65zqnxwxv66k5wel7mljutaq6mt,tokens_in,6.0,6.0,ATOM,0.633418,HUAHUA,124411.558351,8.887505,0.000047,5.629506,5.803674,1,606,11.433180
4,94C1DC72921F5AD1BC0BED6249D5452479DEB62808852D...,4710608,2022-06-07 22:33:38.086,osmo1zzgc9jukqxppns6lu042a87kej5af6f29shel2,tokens_in,6.0,6.0,ATOM,0.003957,HUAHUA,760.271145,8.887505,0.000047,0.035168,0.035466,1,606,0.070634


In [57]:
d = df.groupby(['POOL_ID','MSG_KEY'])['USD'].sum().sort_values(ascending=False).reset_index()

d = d.pivot(index='POOL_ID',columns='MSG_KEY',values='USD').fillna(0)

In [62]:
df.query('MSG_KEY=="tokens_in"')['SENDER'].unique()

array(['osmo1adtxg4klr2z65zqnxwxv66k5wel7mljutaq6mt',
       'osmo1zzgc9jukqxppns6lu042a87kej5af6f29shel2',
       'osmo1lj72jml6hw635cek9hep62u3hvfwng9e2e3r09', ...,
       'osmo1c4sug6r2fdelulq3yl206ds83c4yyxn6lkcchn',
       'osmo16sktkgs03rx60acfxlf7wknykmqjzmf5hfmtd9',
       'osmo177pmvcseh978hhvg8cn2hg4t4emc0ppkf95m04'], dtype=object)

In [78]:
def inandout(ser):
    msg_type = ser['MSG_KEY']
    usd = ser['USD']
    #print(msg_type)
    if 'in' in msg_type:
        return usd
    elif 'out' in msg_type:
        return -1*usd
    else:
        return 0

In [80]:
    df['val'] = df.apply(inandout,axis=1)
    df['val_cum'] = df['val'].cumsum()

In [81]:
df

,TX_ID,BLOCK_ID,BLOCK_TIMESTAMP,SENDER,MSG_KEY,ASSET1_EXPONENT,ASSET2_EXPONENT,ASSET1_SYMBOL,ASSET1_AMOUNT,ASSET2_SYMBOL,ASSET2_AMOUNT,ASSET1_PRICE,ASSET2_PRICE,ASSET1_USD,ASSET2_USD,dummy,POOL_ID,USD,val,val_cum
0,BE7C1F80D781CDA3BF87F3CA2607DEEC9BF3C4DDECBD49...,4709565,2022-06-07 20:45:23.383,osmo1qnqwxazzha6lsx2rs2gewu2337uha5lz5uf2yt,tokens_out,6.0,6.0,CRBRUS,2557.430883,HUAHUA,522.867991,0.000011,0.000047,0.027550,0.024391,1,658,0.051942,-0.051942,-0.051942
1,BC2F40C96EF02403A11B206E77B8EAB2BB0F0F65660D40...,4708075,2022-06-07 18:09:53.084,osmo16mdmh3lfphnmqzn20c59x75nfhfktfz9wexzep,tokens_out,6.0,6.0,CRBRUS,380852.497456,HUAHUA,79554.783095,0.000011,0.000047,4.102815,3.711151,1,658,7.813966,-7.813966,-7.865908
2,DBD856249340F8707CD54DD9C7E2605C9E01A31904A682...,4711580,2022-06-08 00:14:14.320,osmo1wfyaqhxsczc4mlgk54trxznmqkld09p79phzqn,tokens_out,6.0,6.0,ATOM,1.369754,HUAHUA,263978.971856,8.887505,0.000047,12.173696,12.314354,1,606,24.488050,-24.488050,-32.353958
3,2231E194F2EFA3F5818FF6D6E03C13BCEA9FF3312DF6C8...,4712865,2022-06-08 02:27:42.386,osmo1adtxg4klr2z65zqnxwxv66k5wel7mljutaq6mt,tokens_in,6.0,6.0,ATOM,0.633418,HUAHUA,124411.558351,8.887505,0.000047,5.629506,5.803674,1,606,11.433180,11.433180,-20.920778
4,94C1DC72921F5AD1BC0BED6249D5452479DEB62808852D...,4710608,2022-06-07 22:33:38.086,osmo1zzgc9jukqxppns6lu042a87kej5af6f29shel2,tokens_in,6.0,6.0,ATOM,0.003957,HUAHUA,760.271145,8.887505,0.000047,0.035168,0.035466,1,606,0.070634,0.070634,-20.850144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12005,895E5DC531019D8E0C7F2EAED754AE26DD60CA735A3841...,4707870,2022-06-07 17:47:55.824,osmo13fw6wjqngfgnvx3q8lx978jfed6k9rfvjk563f,tokens_in,6.0,NaN,CMDX,0.013566,NaN,NaN,NaN,NaN,NaN,NaN,1,601,NaN,NaN,NaN
12026,E5E88CABE7D5C58C19A6064D369B986C62EE29609F8293...,4708191,2022-06-07 18:22:07.292,osmo177pmvcseh978hhvg8cn2hg4t4emc0ppkf95m04,tokens_in,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,712,NaN,NaN,NaN
12029,67FE0BA131723802603A59F585855F09CAC86BA7C08831...,4709166,2022-06-07 20:03:52.131,osmo17329x9x02qepkaqfx8rpyvn2x0pp5v3f7w3qtt,tokens_in,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,712,NaN,NaN,NaN
12060,235BB8BD4FA24D3CEE8DE649CA8FFEB522D11272749547...,4712246,2022-06-08 01:23:31.448,osmo1espd3tmahfpe7074nqhzulcu5fdvhm48trrg3t,tokens_in,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,678,NaN,NaN,NaN
